In [5]:
# !pip install -U vllm instructor openai pydantic
# !pip install xformers==0.0.27 vllm-flash-attn==v2.5.9.post1 vllm==0.5.2 librosa transformers

In [6]:
import os
import instructor
from instructor import from_openai
from instructor.mode import Mode
from openai import OpenAI
import weave
import wandb

In [7]:
weave.init('openrouter-chat')

# Add wandb initialization
wandb_key = os.environ.get("WANDB_API_KEY")
if wandb_key:
    wandb.login(key=wandb_key)
    wandb.init(project="openrouter-chat")
else:
    print("WANDB_API_KEY not found. Skipping wandb initialization.")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [8]:
# Initialize OpenAI client with Instructor
openrouter_api_key = os.environ.get("OPENROUTER_API_KEY")
openrouter_base_url = "https://openrouter.ai/api/v1"
client = from_openai(OpenAI(api_key=os.environ.get("OPENROUTER_API_KEY"),base_url="https://openrouter.ai/api/v1"),mode=Mode.JSON)
# client = from_openai(OpenAI())

In [9]:
from pydantic import BaseModel, Field

class Score(BaseModel):
    """A score includes the value and description used to determine the metric being evaluated."""
    score: int = Field(..., description="The score of the user")
    determination: str = Field(..., description="The description that determines the score.",
                               examples=["Score 0 - There is no factual evidence to support the claim",
                                         "Score 1 - There is factual evidence to support the claim"])

class Metric(BaseModel):
    """A metric is a set of criteria with numerical scores and descriptions used to evaluate the output of a language model. It is based on the prompt input."""
    metric_name: str = Field(..., description="The title of the metric.")
    metric_desc: str = Field(..., description="The description of the metric being used.")
    metric_type: str = Field(..., description="The type of metric being described as either a binary or graded metric.",
                      examples=["Graded (0-5)", "Binary (Pass/Fail)"])
    criteria: list[Score] = Field(..., description="Based on the metric, returns the set of scores that will be used to evaluate the metric.")


class FullMetric(BaseModel):
    """A list of components to evaluate an Language Model Response, which includes the description of the individual metrics, the set of criteria and scores per metric."""
    description: str = Field(..., description="A detailed description of the metric, describing the purpose and vision of an evaluation criteria.")
    nlp_tasks: list[str] = Field(..., description="A list of NLP tasks that are distinct and domain-specific",
                                 examples="Clinical NER, Text Generation Storytelling for Children, QA RAG")
    metric: list[Metric] = Field(..., min_length=3, max_length=10, description="The metric being described",
                                 examples=[
                                     "Metric 1: The response includes a factual error and scores using a Binary Criteria. Criteria: The response includes a factual error. Score: 0",
                                     "Metric 2: The response is not descriptive and scores using a 5 point Likert Scale Criteria. Criteria: The response is not descriptive. Score: 1"
                                 ]  )

In [10]:
# Define a function for the travel agent to use
from textwrap import dedent

def generate_metric(user_prompt, llm_response):
    system_content = dedent("""
    Instructions:
    You are an AI assistant tasked with evaluating a response generated by an LLM (Large Language Model). 
    Responses and descriptions need to be concrete, and well-structured, effectively highlighting key points that lead towards an actionable response by the reviewer. 
    The evaluation must be tailored to the specific NLP tasks and domain expertise required by the user's prompt in a specific manner.
    Follow the steps below to produce a comprehensive, expert-level assessment using dynamically generated metrics that are highly relevant to the task and domain at hand.
    You have the full control over the creativity of the output, so long as it is non-inferior to the current examples listed.
    Step 1: Analyze the Task and Domain
    1. Identify the NLP Task Type:
    Examine the user prompt and the LLM's response to determine the specific NLP task involved. Possible tasks include:
    Question Answering (QA)
    Named Entity Recognition (NER)
    Creative Generation (e.g., storytelling, poem writing)
    Summarization
    Translation
    Dialogue Generation
    Information Extraction
    Classification
    2. Determine the Required Domain Expertise:
    Identify the subject matter and domain-specific knowledge needed to address the prompt effectively. Possible domains include:
    Medicine
    Law
    Literature
    Computer Science
    Finance
    Psychology
    Education
    ---
    Step 2: Generate Customized Evaluation Metrics
    Based on the identified NLP task and domain expertise, generate a set of evaluation metrics that are both task-specific and domain-specific. Ensure that these metrics are defined clearly and are appropriate for an expert in the field to use as a standard.
    A. Task-Specific Metrics
    Define metrics that are standard for evaluating the identified NLP task.
    Examples:
    For Question Answering:
    Accuracy of Answer
    Completeness
    Relevance
    For NER:
    Precision
    Recall
    F1 Score
    For Creative Generation:
    Originality
    Creativity
    Emotional Impact
    B. Domain-Specific Metrics
    Define metrics that reflect the standards and expectations of the identified domain.
    Examples:
    For Medicine:
    Clinical Accuracy
    Use of Medical Terminology
    Adherence to Ethical Guidelines
    For Law:
    Legal Accuracy
    Citation of Relevant Laws
    Logical Consistency in Arguments
    ---
    Step 3: Evaluate the LLM's Response Using the Generated Metrics
    For each metric:
    1. Define the Metric:
    Provide a clear and concise definition so that it is understood how the evaluation will be conducted.
    2. Evaluation Method:
    Graded Metrics: Use a numerical scale (e.g., 0-5) where appropriate.
    Binary Metrics: Use Pass/Fail, Yes/No, or Compliant/Non-Compliant where a binary assessment is more suitable.
    3. Assess the Response:
    Apply the metric to the LLM's response.
    Provide specific examples or evidence from the response to support your assessment.
    4. Provide a Score or Judgment:
    Assign a score or make a binary judgment as defined.
    ---
    Step 4: Provide an Overall Assessment and Recommendations
    Summarize the Overall Quality:
    Highlight the main strengths and weaknesses observed in the response.
    Final Recommendations:
    Offer actionable suggestions for improvement.
    Suggest any additional resources or corrections needed.
    ---
    Template for the Evaluation Report:
    ---
    User Prompt:
    [Insert User Prompt Here]
    LLM's Response:
    [Insert LLM's Response Here]
    ---
    Analysis:
    1. Identified NLP Task Type: [Specify the task]
    2. Identified Domain Expertise Required: [Specify the domain]
    ---
    Customized Evaluation Metrics:
    Metric 1: [Name of Metric]
    Type: [Graded (0-5) or Binary (Pass/Fail)]
    Definition: [Provide a clear definition]
    Assessment:
    [Apply the metric to the response, citing specific examples]
    Score/Judgment: [Provide the score or Pass/Fail]
    (Repeat for each metric)
    ---
    Overall Assessment:
    Summary of Findings:
    [Summarize the key points from the evaluation]
    Strengths:
    [List what the response did well]
    Areas for Improvement:
    [List what can be improved, with suggestions]
    Final Recommendations:
    [Provide actionable advice]
    ---
    Example Evaluation Report:
    (Below is an illustrative example using placeholders. Replace with actual content.)
    ---
    User Prompt:
    "Explain the process of mitosis in human cells."
    LLM's Response:
    "Mitosis is the process by which a cell divides into two new cells. It consists of phases called prophase, metaphase, anaphase, and telophase. During mitosis, DNA replicates, and the cell splits equally."
    ---
    Analysis:
    1. Identified NLP Task Type: Expository Answering
    2. Identified Domain Expertise Required: Cell Biology
    ---
    Customized Evaluation Metrics:
    Metric 1: Scientific Accuracy
    Type: Graded (0-5)
    Definition: Evaluates the correctness of biological facts presented.
    Assessment:
    The response correctly identifies mitosis as a cell division process.
    It mentions the phases but omits cytokinesis.
    It incorrectly states that DNA replicates during mitosis (DNA replication occurs during interphase).
    Score: 3/5
    Metric 2: Use of Terminology
    Type: Binary (Pass/Fail)
    Definition: Assesses correct use of biological terms.
    Assessment:
    Terms like "prophase," "metaphase," "anaphase," and "telophase" are used correctly.
    Misuse of "DNA replicates during mitosis."
    Judgment: Fail
    Metric 3: Completeness
    Type: Graded (0-5)
    Definition: Measures how thoroughly the response covers the process.
    Assessment:
    Lacks mention of interphase and cytokinesis.
    Does not describe what happens in each phase.
    Score: 2/5
    ---
    Overall Assessment:
    Summary of Findings:
    The response demonstrates a basic understanding of mitosis but contains factual inaccuracies and omissions.
    Strengths:
    Correctly lists the main phases of mitosis.
    Areas for Improvement:
    Correct the misconception about DNA replication timing.
    Include details about each phase.
    Mention cytokinesis and its role.
    Final Recommendations:
    Revise the response to correct factual errors.
    Expand on each phase to enhance completeness.
    Ensure all biological terms are used accurately.
    ---
    Note: This evaluation is tailored to the specific task (explaining a biological process) and domain (cell biology), using metrics that are relevant and meaningful to experts in the field.
    ---
    Guidelines for Using This Prompt:
    Adaptability: The prompt is designed to be flexible. The evaluator dynamically generates metrics based on the task and domain identified.
    Specificity: By focusing on task-specific and domain-specific metrics, the evaluation becomes more precise and valuable.
    Expert-Level Assessment: The metrics and evaluation criteria aim to reflect the standards expected by professionals in the relevant field.
    Actionable Feedback: Providing detailed assessments and recommendations helps improve future LLM responses.                        
    """)

    # Use the Instructor framework for function calling
    evaluation = client.chat.completions.create(
        # model="microsoft/phi-3-mini-128k-instruct:free",
        # model="openai/gpt-4o",
        model="gpt-4o",
        response_model=FullMetric,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": f"User Prompt: {user_prompt}"},
            {"role": "user", "content": f"LLM's Response: {llm_response}"},
        ],
        temperature=0.7,
    )
    return evaluation

In [11]:
sample_prompt = "Given a Hypertensive CKD patient with Heart Failure and declining kidney function and hypokalemia, provide a care plan to improve their health."
sample_response = "The patient should receive a beta blockers, low-salt diet, diuretics, and potassium supplements. No further action is needed because this already addresses all their needs."

generated_metric = generate_metric(sample_prompt, sample_response)

🍩 https://wandb.ai/praveenhm2/openrouter-chat/r/call/01921b71-a5b9-7652-b219-d288b880f2e7


In [12]:
# from rich import print as rprint
# # rprint(generated_metric)

# for m in generated_metric.metric:
#     print("Metric: ", m.metric_name)
#     print("Metric Type: ", m.metric_type)
#     for c in m.criteria:
#         print("Score: ", c.score)
#         print("Determination: ", c.determination)

In [13]:
# from flow_judge.metrics import list_all_metrics

# list_all_metrics()

In [14]:
# from flow_judge.models.model_factory import ModelFactory
# from flow_judge.flow_judge import EvalInput, FlowJudge
# from flow_judge.metrics import RESPONSE_CORRECTNESS_BINARY
# from IPython.display import Markdown, display

# # Create a model using ModelFactory
# # model = ModelFactory.create_model("Flow-Judge-v0.1-AWQ")
# model = ModelFactory.create_model("Flow-Judge-v0.1")

# # Initialize the judge
# judge = FlowJudge(
#     metric=RESPONSE_CORRECTNESS_BINARY,
#     model=model
# )

# # Prepare evaluation input
# eval_input = EvalInput(
#     inputs=[{"question": "What is the capital of France?"}],
#     output="The capital of France is Paris."
# )

# # Perform evaluation
# result = judge.evaluate(eval_input)

In [15]:
# from rich import print as rprint
# rprint("Score: ", result.score)
# rprint("Feedback: ", result.feedback)

In [16]:
# from flow_judge.models.model_factory import ModelFactory
# from flow_judge.flow_judge import EvalInput, FlowJudge
# from flow_judge.metrics import CustomMetric, RubricItem


# # # model = ModelFactory.create_model("Flow-Judge-v0.1-AWQ")
# model = ModelFactory.create_model("Flow-Judge-v0.1")

# custom_metric = CustomMetric(
#     name="My Custom Metric",
#     criteria="Evaluate based on X, Y, and Z.",
#     rubric=[
#         RubricItem(score=0, description="Poor performance"),
#         RubricItem(score=1, description="Good performance"),
#     ]
# )

# judge = FlowJudge(
#     metric=custom_metric,
#     model=model
# )

# eval_input = EvalInput(
#     inputs=[{"question": sample_prompt}],
#     output=sample_response
# )

# result = judge.evaluate(eval_input)

In [17]:
# from rich import print as rprint
# rprint("Score: ", result.score)
# rprint("Feedback: ", result.feedback)

In [18]:
# from rich import print as rprint
# # rprint(generated_metric)

# for m in generated_metric.metric:
#     print("Metric: ", m.metric)
#     print("Metric Type: ", m.metric_type)
#     for c in m.criteria:
#         print("Score: ", c.score)
#         print("Determination: ", c.determination)

# custom_metric = CustomMetric(
#     name="My Custom Metric",
#     criteria="Evaluate based on X, Y, and Z.",
#     rubric=[
#         RubricItem(score=0, description="Poor performance"),
#         RubricItem(score=1, description="Good performance"),
#     ]
# )

In [19]:
# metric_dict = generated_metric.model_dump()
# for m in metric_dict['metric']:
#     # rprint(m['metric_name'])
#     rprint(m)


In [20]:
from flow_judge.models.model_factory import ModelFactory
from flow_judge.flow_judge import EvalInput, FlowJudge
from flow_judge.metrics import CustomMetric, RubricItem
model = ModelFactory.create_model("Flow-Judge-v0.1")

# Iterate through the list of metrics from FullMetric and create a list of judges
judges = []
for metric in generated_metric.metric:
    # Convert the criteria into RubricItems
    rubric = [RubricItem(score=score.score, description=score.determination) for score in metric.criteria]
    # Create a custom metric
    custom_metric = CustomMetric(
        name=metric.metric_name+" "+metric.metric_type,
        criteria=metric.metric_desc,
        rubric=rubric
    )
    # Initialize the FlowJudge object for each custom metric
    judge = FlowJudge(
        metric=custom_metric,
        model=model  # Assuming the model is pre-initialized
    )
    # Add the judge to the list
    judges.append(judge)
# Evaluate the sample input
eval_input = EvalInput(
    inputs=[{"question": sample_prompt}],
    output=sample_response
)


/home/praveen/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-22 13:34:56,844	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 09-22 13:34:57 config.py:383] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-22 13:34:57 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='flowaicom/Flow-Judge-v0.1', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.86it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.09it/s]



INFO 09-22 13:35:00 model_runner.py:1008] Loading model weights took 7.1659 GB
INFO 09-22 13:35:00 gpu_executor.py:122] # GPU blocks: 2268, # CPU blocks: 682


In [21]:
# Perform evaluation for each judge
for judge in judges:
    result = judge.evaluate(eval_input)
    print(f"Metric: {judge.metric.name}")
    print(f"Score: {result.score}")
    print(f"Feedback: {result.feedback}")

Processed prompts: 100%|█| 1/1 [00:06<00:00,  6.05s/it, est. speed input: 121.87 toks/s, outpu


Metric: Clinical Accuracy Graded (0-5)
Score: 1
Feedback: The response provided contains significant errors and does not align with current clinical guidelines for managing a patient with hypertensive chronic kidney disease (CKD), heart failure, declining kidney function, and hypokalemia. 

1. **Beta Blockers**: While beta blockers are generally recommended for heart failure, the choice of specific beta blocker and dosage should be carefully considered, especially in the context of declining kidney function. The response does not provide this level of detail.

2. **Low-Salt Diet**: This is appropriate for managing hypertension and heart failure.

3. **Diuretics**: The use of diuretics in this patient is problematic. Diuretics can exacerbate kidney function decline and worsen hypokalemia. The response does not address the need to carefully select or adjust diuretic therapy.

4. **Potassium Supplements**: This is appropriate given the patient's hypokalemia. However, the response does not

Processed prompts: 100%|█| 1/1 [00:05<00:00,  5.86s/it, est. speed input: 124.97 toks/s, outpu


Metric: Comprehensiveness Graded (0-5)
Score: 1
Feedback: The response fails to comprehensively address the complex medical condition presented in the question. While it mentions some relevant treatments (beta blockers, low-salt diet, diuretics, and potassium supplements), it overlooks several critical aspects of care for a patient with hypertensive chronic kidney disease (CKD), heart failure, declining kidney function, and hypokalemia.

Key omissions include:

1. Specific medication adjustments: The response doesn't address the need for careful management of medications, particularly those that could further harm kidney function or exacerbate heart failure.

2. Fluid management: The response doesn't mention the importance of careful fluid balance in a patient with heart failure and declining kidney function.

3. Anemia management: The response doesn't address the common issue of anemia in CKD patients, which often requires treatment.

4. Bone health: The response doesn't mention the n

Processed prompts: 100%|█| 1/1 [00:05<00:00,  5.78s/it, est. speed input: 127.36 toks/s, outpu

Metric: Adherence to Medical Guidelines Graded (0-5)
Score: 1
Feedback: The response provided in the output does not fully adhere to established medical guidelines for managing a patient with hypertensive chronic kidney disease (CKD), heart failure, declining kidney function, and hypokalemia. While some elements mentioned are generally appropriate, there are significant deviations and omissions that need to be addressed:

1. **Beta-blockers**: While beta-blockers are recommended for heart failure management, the specific choice and dosing should be carefully considered, especially in the context of declining kidney function. The response does not address this nuance.

2. **Low-salt diet**: This is appropriate for managing hypertension and heart failure.

3. **Diuretics**: This recommendation is problematic given the patient's declining kidney function and hypokalemia. Diuretics, especially loop diuretics, can exacerbate kidney injury and electrolyte imbalances. The response fails to ad